In [1]:
import os
import mykey
os.environ['OPENAI_API_KEY'] = mykey.openai()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

In [2]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
examples = [
    {
        "question" : """다음 내용들로 명훈 스타일 공지를 작성해주세요. 
            날짜 : 1월 24일 
            시간 : 없음
            컨텐츠 : 휴방 
            할 말 : 산악회 로아 3막 레이드 완료""",
        
        "answer" : """[공지] 1월 24일 명예훈장 방송 공지입니다! 
        반갑습니다 사랑하는 턱붕이 여러분!! 으아 로아 이번 3막 엄청 빡셌다 ㅠㅠ!!!!  
        그래도 2일만에 클리어 완료.... ㅋㅋㅋ 다들 응원해주셔서 감사합니다!  
        전체 순위는 6등이네요  
        후후 잔혈까지 나이스 +_+ 스펙업 한 보람이 있었습니다 도와주신분들 너무너무x999 감사합니다ㅎ!  
        산악회 다들 고생많았다!!!  
        저는 오늘 배터리 방전입니다 일좀 하면서 쉬도록 하겠습니다 ㅎ  
        유튜브 또 올라왔으니 재밌게 봐주시고 다시보기 채널도 많이 사랑해주세요!  
        항상 시청해주셔서 감사합니다! 좋은 저녁 보내세요!"""
    },
    {
        "question" : """다음 내용들로 명훈 스타일 공지를 공지 작성해주세요.  
            날짜 : 1월 23일  
            시간 : 3시, 7시  
            컨텐츠 : 명조 광고, 로아 3막 트라이  
            할 말 : 3시 명조 광고, 26시간 진행한 로아 3막 리트라이""",
        "answer" : """[공지] 반갑습니다 1월 23일 방송 공지입니다! 3시 - 명조 광고 / 7시 로아 산악회 리트라이 가자!  
        반갑습니다 사랑하는 턱붕이 여러분!! 1월 23일 방송 공지입니다!  
        오우 어제 26시간 트라이....하고 잠깐 자고 왔습니다ㅋㅋㅋ 아직 3막을 못 깼다.....  
        오늘 아침에 갑자기 정상화 패치를 해서 후반 진도를 나가야되는데 초반 진도를 나가게 되었습니다 크흡..  
        방송 일정 공지!!  
        3시 : 명조 광고!!!! 제가 요즘 진짜 재밌게 하는 모바일게임 띵조에서 연락이 왔습니다 와ㅏㅏ  
        오늘 새로운 캐릭터 로코코도 나왔다고 하는데 카멜리아랑 같이 쓴다고 해서 한번 뽑아보겠습니닷!!!!!  
        7시 : 산악회 로아 3막 리트라이  
        이렇게 예정되어있습니다!  
        오늘 유튭 컨펌 빠르게 하고 유튭 영상도 올라올 예정입니다 영상 올라오면 좋아요 마니 눌러주세용! """
    },
    {
        "question" : """다음 내용들로 명훈 스타일 공지 작성해주세요.  
            날짜 : 1월 20일  
            시간 : 8시  
            컨텐츠 : 로아 산악회, 메이플 림보, 던파 태초 런  
            할 말 : 로아 상상악세 스펙업""",
        "answer" : """1월 20일 명훈 방송 공지! 오늘 로아 스펙업 완료!! 메던로의 날이다!!  
            1월 20일 명훈 방송 공지!  
            오늘 로아 스펙업 완료 상상 악세 + 상상+@ 맞춰왔습니다!  
            명튜브도 업로드했으니 좋아요+재밌게 시청해주세요!  
            메던로의 날이다!!  
            1부 로아 산악회  
            2부 메이플 림보 ㄱㄱ  
            3부 던파 태초 도파민 중독자 출격ㅋㅋ  
            이따 봅시다 오늘 8시 거의 딱 맞춰서 올듯용?  
            """
    },
    {
        "question" : """다음 내용들로 명훈 스타일 공지 작성해주세요.  
            날짜 : 1월 19일  
            시간 : 5시 10분  
            컨텐츠 : 메이플 검은마법사  
            합방멤버 : 메벤저스 
            할 말 : 메벤저스 컨텐츠 메카닉 검은마법사 마무리""",
        "answer" : """1월 19일 방송 명훈 스타일 공지.. 오늘 메벤져스 라스트댄스! 
            반갑습니다 사랑하는 턱붕이 여러분!  
            1월 19일 명훈 방송 공지입니다!  
            오늘 5시 10분 방송 ON~~ (메벤져스 5시 30분 스타트입니다!)  
            후 어제 열심히 달리고 자고 왔씁니다 오늘 메벤져스 라스트댄스 검은마법사 쭉 달려보자구!!  
            사실상 오늘이 마지막.. 아름다운 마무리가 되길ㅋㅋㅋㅋ  
            메카닉 정 많이 들었는데 이제 떠나보낼때가 됐네요..  
            이따 봅시다!ㅎ"""
    },
    
]

In [4]:
example_prompt = PromptTemplate.from_template(
    "Question: n{question} nAnswer: n{answer}"
)

print(example_prompt.format(**examples[0]))


Question: n다음 내용들로 명훈 스타일 공지를 작성해주세요. 
            날짜 : 1월 24일 
            시간 : 없음
            컨텐츠 : 휴방 
            할 말 : 산악회 로아 3막 레이드 완료 nAnswer: n[공지] 1월 24일 명예훈장 방송 공지입니다! 
        반갑습니다 사랑하는 턱붕이 여러분!! 으아 로아 이번 3막 엄청 빡셌다 ㅠㅠ!!!!  
        그래도 2일만에 클리어 완료.... ㅋㅋㅋ 다들 응원해주셔서 감사합니다!  
        전체 순위는 6등이네요  
        후후 잔혈까지 나이스 +_+ 스펙업 한 보람이 있었습니다 도와주신분들 너무너무x999 감사합니다ㅎ!  
        산악회 다들 고생많았다!!!  
        저는 오늘 배터리 방전입니다 일좀 하면서 쉬도록 하겠습니다 ㅎ  
        유튜브 또 올라왔으니 재밌게 봐주시고 다시보기 채널도 많이 사랑해주세요!  
        항상 시청해주셔서 감사합니다! 좋은 저녁 보내세요!


In [5]:
example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Question: n{question} nAnswer: n{answer}')

In [6]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: \n {question} \n Answer:",
    input_variables=["question"],
)

In [12]:
mquestion = "다음 내용들로 명훈 스타일 공지를 작성해주세요. 날짜 : 1월 31일,  시간 : 8시, 컨텐츠 : 산악회, 메이플 영도재획"
print(mquestion)

다음 내용들로 명훈 스타일 공지를 작성해주세요. 날짜 : 1월 31일,  시간 : 8시, 컨텐츠 : 산악회, 메이플 영도재획


In [13]:
prompt.input_variables

['question']

In [14]:
final_prompt = prompt.format(question=mquestion)

In [15]:
print(model.invoke(final_prompt).content)

[공지] 1월 31일 명훈 방송 공지! 산악회와 메이플 영도재획으로 즐거운 시간 보내요!  
반갑습니다 사랑하는 턱붕이 여러분!  
1월 31일 명훈 방송 공지입니다!  
오늘 8시 시작되는 방송에서는 산악회와 메이플 영도재획으로 즐거운 시간을 보내보려고 합니다!  
산악회에서는 함께 모험을 떠나고, 메이플에서는 영도재획을 통해 다양한 보물을 획득해보도록 하겠습니다!  
이따 보시면서 즐거운 시간 보내시고, 항상 응원해주셔서 감사합니다! 함께해요~!
